In [ ]:
from helper import *
from dataloader import *
from train_val import *

In [ ]:
iin=random.randrange(len(train_df))
inspect_dataset(iin,'train')

print(len(train_df),len(test_df))

In [ ]:
trainset = PersonSegmentationDataset(train_df,get_train_augs())
validset = PersonSegmentationDataset(test_df,get_val_augs())
print(f"Size of Trainset : {len(trainset)}")
print(f"Size of Validset : {len(validset)}")

In [ ]:
inn=random.randrange(len(trainset))
check_dataloaders(index=inn,loader='trainset')

In [ ]:
from torch.utils.data import DataLoader
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
validloader =  DataLoader(validset,batch_size=BATCH_SIZE,)

#trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True,generator=torch.Generator(device='cuda'))
#validloader =  DataLoader(validset,batch_size=BATCH_SIZE,generator=torch.Generator(device='cuda'))
print(f'Total number of batches in train loader : {len(trainloader)}')
print(f'Total number of batches in Valid loader : {len(validloader)}')

In [ ]:
for images,mask in trainloader:
  print(f' One batch image shape {images.shape}')
  print(f' One batch mask shape {mask.shape}')
  break

In [ ]:
import torch.nn as nn
from segmentation_models_pytorch import Unet
from monai.losses import DiceLoss

In [ ]:
class DRSegmentationModel(nn.Module):
  def __init__(self):
    super(DRSegmentationModel, self).__init__()
    self.arc = Unet(encoder_name=ENCODER,
                        encoder_weights=WEIGHTS,
                        in_channels=3,
                        classes=1,
                        activation=None,
                        )
  def forward(self, images, masks=None):
    logits = self.arc(images)
    if masks!=None:
      loss1=DiceLoss(mode='binary')(logits, masks)
      loss2= nn.BCEWithLogitsLoss()(logits, masks)
      return logits, loss1+loss2

    return logits

In [ ]:
model=DRSegmentationModel()
model.to(DEVICE)

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingLR, ReduceLROnPlateau
# from torch.cuda.amp import autocast, GradScaler
optimizer = torch.optim.Adam(model.parameters(), lr =1e-4, weight_decay =1e-5)
scheduler = OneCycleLR(optimizer, max_lr=1e-3, epochs=EPOCHS, steps_per_epoch=len(trainloader)) 
# scaler=GradScaler()

In [ ]:
best_valid_loss=np.Inf
early_stp=0
for i in range (EPOCHS):
  train_loss=train_net(trainloader, model, optimizer)
  valid_loss= eval_net(validloader, model)
  scheduler.step()  #Lr schedulers
  if valid_loss <best_valid_loss:
    torch.save(model.state_dict(), 'UNet-vgg16_model_micro_adam50.pt')
    print("Model saved")
    best_valid_loss=valid_loss
    early_stp=0
  else:
    early_stp+=1
    if early_stp>=8:
      print("Eraly stopping")
      break
  print(f'ECPOCH : {i+1} Training Loss : {train_loss}, Validation Loss : {valid_loss}')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


In [ ]:
model.load_state_dict(torch.load('/home/abir/Documents/PROJECTS/seg_folder/UNet-vgg16_model_micro_adam50.pt')) # its 80+

In [ ]:
# index=random.randrange(len(validset))
# model.load_state_dict(torch.load('/home/abir/Documents/PROJECTS/seg_folder/saved models/best_model_micro_adam.pt'))

# model=model.to(DEVICE)
for i in range(len(validset)):
    image, mask = validset[i]
    logits_mask = model(image.to(DEVICE).unsqueeze(0)) # (c, h, w) ---> (1, C, H,W) , 1 for batch
    pred_mask= torch.sigmoid(logits_mask)  # as we have not used sigmoid in pred function
    pred_mask = (pred_mask>0.5)*1.0   # convert any pixel less greater than .5 to 1

    show_image(image, mask, pred_mask.detach().cpu().squeeze(0)) # detach and remove batch info added earlier  
    pred_mask=pred_mask.detach().cpu()
    iou=iou_(mask,pred_mask)
    f1score=f1_score_(mask,pred_mask)
    # pred_mask=(pred_mask>0.5).float()
    # prd_mask=pred_mask.detach().cpu().numpy().flatten()                                            #.astype(int)
    # msk=mask.detach().cpu().numpy().flatten()                                                      #.astype(int)

    # iou=jaccard_score(y_true=msk,y_pred=prd_mask)
    # F1_score =f1_score(y_true=msk,y_pred=prd_mask)

    print(f" IOU:{iou} || F1 score:{f1score}")
    # print(f"Recall:{recall} Precision:{precision}")